In [1]:
sents = ['Переехав в Парк культуры, «Гараж» сначала квартировал во временном здании японского архитектора Сигеру Бана, а затем укоренился в еще одном общепите — реконструированном Рэмом Колхасом ресторане «Времена года».',\
         'Ходили слухи, что «Шестигранником» тоже займется великий голландец.',\
         'Но он за это время успел взяться ремонтировать Новую Третьяковку.',\
         'В декабре рядышком, буквально через реку, наконец-то откроется «ГЭС-2», еще один культурно значимый частный проект — пятого форбса Леонида Михельсона.',\
         'В Москве сегодня много чего происходит: скоро откроется Политех, достраиваются Средние торговые ряды у Красной площади (тут будет новый выставочный комплекс Музеев Кремля).',\
         'В следующем году закончится строительство Новой Третьяковки в Коммунарке, свое хранилище доделывает и Пушкинский музей.',\
         'Более того, в сентябре четыре музея — федеральные Пушкинский и Третьяковка и частные «Гараж» и «ГЭС-2» — объединились в «музейную четверку» по примеру лондонских коллег.',\
         'Будут не просто сообща окультуривать «прилегающую территорию», а придумывать совместные программы и всячески пропагандировать друг друга.',\
         'Антон Белов цитирует известного инвестора Рэя Далио, чью книгу недавно перевели на русский: «Его спросили, в чем проблема России. И он сказал: «Слушайте, вам просто надо объединяться и общаться между собой».',\
         'C 2018 года здесь развивают программы патронажа, осенью проведут первый ежегодный гала-ужин эндаумент-фонда.',\
         'Объем эндаумент-фонда музея "Гараж" составляет 570 438 599 рублей.',\
         'Для сравнения, фонд целевого капитала Музея Метрополитен в Нью-Йорке составляет около 3 миллиардов долларов.',\
         'В России закон «О порядке формирования и использования целевого капитала некоммерческих организаций» был принят в 2006 году.']

In [15]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

ne = NamesExtractor(morph_vocab) #возвр инфо про место начала и конце ne, first и last name
ae = AddrExtractor(morph_vocab) #возвр инфо про место начала и конце ne, отдельно составные части
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)

from ipymarkup import show_span_line_markup

Напишем функцию для изначальной разметки по умолчанию:

In [3]:
def ner_by_default(sent):
    ents = list(ne(sent)) + list(ae(sent))
    prepare_ents = [] # список туплов со стартовой позицией ne/ae и названием вида сущности
    for ent in ents: 
        prepare_ents.append((
          ent.start,
          ent.stop,
          ent.fact.__class__.__name__
        ))
    return show_span_line_markup(
    sent,
    prepare_ents
    )

In [4]:
for sent in sents:
    ner_by_default(sent)

Задание 1. Отсмотрев результаты извлечения, написать правила фильтрации ложно-положительных сущностей - 1.5 балла

Посмотрим, какие теги приписаны каким словам, чтобы понять, что нужно исправить:

In [5]:
for sent in sents[:2]:
    ents = list(ne(sent)) + list(ae(sent))
    for ent in ents:
        print(ent)

Match(start=9, stop=10, fact=Name(first=None, last='в', middle=None))
Match(start=27, stop=32, fact=Name(first=None, last='Гараж', middle=None))
Match(start=54, stop=56, fact=Name(first=None, last='во', middle=None))
Match(start=57, stop=66, fact=Name(first=None, last='временном', middle=None))
Match(start=84, stop=95, fact=Name(first=None, last='архитектора', middle=None))
Match(start=96, stop=107, fact=Name(first='Сигеру', last='Бана', middle=None))
Match(start=109, stop=110, fact=Name(first=None, last='а', middle=None))
Match(start=128, stop=129, fact=Name(first=None, last='в', middle=None))
Match(start=134, stop=139, fact=Name(first=None, last='одном', middle=None))
Match(start=170, stop=184, fact=Name(first='Рэмом', last='Колхасом', middle=None))
Match(start=7, stop=12, fact=Name(first=None, last='слухи', middle=None))
Match(start=49, stop=56, fact=Name(first=None, last='великий', middle=None))


Нам бы надо сделать так, что что-то в кавычках с большой буквы, это тоже Name, а что с маленькой буквы, это не Name. Напишем функцию:

In [6]:
def custom_ner(sent):
    ents_ne = list(ne(sent))
    prepare_ents = []
    for ent in ents_ne:
        f = ent.fact.first
        l = ent.fact.last
        m = ent.fact.middle
        if (f and f[0].isupper()) or (l and l[0].isupper()) or (m and m[0].isupper()):
            prepare_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__
            ))
    ents_ae = list(ae(sent))
    for ent in ents_ae:
        v = ent.fact.value
        if v and v[0].isupper():
            prepare_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__
            ))
    show_span_line_markup(
    sent,
    prepare_ents
    )

Задание 2. Визуализировать на N примерах до и после (при этом сравнивать результаты должно быть просто и на глаз) - 1 балл  
Сравним результаты изначальной разметки и результаты улучшенной разметки:

In [7]:
for sent in sents:
    ner_by_default(sent)
    print('-')
    custom_ner(sent)
    print('*'*8)

-


********


-


********


-


...

********


-


********


-


********


Задание 3. Написать функцию сортировки списка сущностей:

*   использовать полный перебор в дальнейшем  - 0 баллов
*   python sort + lambda - 1 балл
*   custom sort - 3 балла

Для начала напишем функцию, которая будет отдельно возвращать список характеристик сущностей:

In [8]:
def get_list_of_ents(sent):
    ents_ne = list(ne(sent))
    list_of_ents = []
    for ent in ents_ne:
        f = ent.fact.first
        l = ent.fact.last
        m = ent.fact.middle
        if (f and f[0].isupper()) or (l and l[0].isupper()) or (m and m[0].isupper()):
            list_of_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__
            ))
    ents_ae = list(ae(sent))
    for ent in ents_ae:
        v = ent.fact.value
        if v and v[0].isupper():
            list_of_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__
            ))
    return list_of_ents

Далее напишем алгоритм сортировки вставкой для такого списка кортежей из характеристик сущностей:

In [9]:
def sort_with_insertion(list_of_ents):
    for i in range(1,len(list_of_ents)):
        j = i-1
        key = list_of_ents[i]
        while list_of_ents[j][0] > key[0] and j >= 0:
            list_of_ents[j+1] = list_of_ents[j]
            j -= 1
        list_of_ents[j+1] = key
    return list_of_ents

sort_with_insertion(get_list_of_ents(sents[0]))

[(27, 32, 'Name'), (96, 107, 'Name'), (170, 184, 'Name')]

In [10]:
for sent in sents:
    print(sent, '\n', sort_with_insertion(get_list_of_ents(sent)))

Переехав в Парк культуры, «Гараж» сначала квартировал во временном здании японского архитектора Сигеру Бана, а затем укоренился в еще одном общепите — реконструированном Рэмом Колхасом ресторане «Времена года». 
 [(27, 32, 'Name'), (96, 107, 'Name'), (170, 184, 'Name')]
Ходили слухи, что «Шестигранником» тоже займется великий голландец. 
 []
Но он за это время успел взяться ремонтировать Новую Третьяковку. 
 [(47, 52, 'Name')]
В декабре рядышком, буквально через реку, наконец-то откроется «ГЭС-2», еще один культурно значимый частный проект — пятого форбса Леонида Михельсона. 
 [(0, 1, 'Name'), (131, 149, 'Name')]
В Москве сегодня много чего происходит: скоро откроется Политех, достраиваются Средние торговые ряды у Красной площади (тут будет новый выставочный комплекс Музеев Кремля). 
 [(0, 1, 'Name'), (2, 8, 'Name'), (2, 8, 'AddrPart'), (79, 86, 'Name'), (103, 110, 'Name'), (103, 118, 'AddrPart'), (164, 170, 'Name')]
В следующем году закончится строительство Новой Третьяковки в Коммуна

Задание 5. Добавить еще экстраткоры из наташи (минимум +2 экстратора) и дописать правила фильтрации (если нужны) и правила структурирования - 1.5 балла

Добавим dates_extractor и money_extractor ко всему что было до:

In [11]:
for sent in sents:
    ents_ne = list(ne(sent))
    prepare_ents = []
    for ent in ents_ne:
        f = ent.fact.first
        l = ent.fact.last
        m = ent.fact.middle
        if (f and f[0].isupper()) or (l and l[0].isupper()) or (m and m[0].isupper()):
            prepare_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__
            ))
    ents_ae = list(ae(sent))
    for ent in ents_ae:
        v = ent.fact.value
        if v and v[0].isupper():
            prepare_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__
            ))
    ents_dt_mn = list(dates_extractor(sent)) + list(money_extractor(sent))
    for ent in ents_dt_mn: 
        prepare_ents.append((
          ent.start,
          ent.stop,
          ent.fact.__class__.__name__
        ))
    show_span_line_markup(
    sent,
    prepare_ents
    )

Зафиналим тем, как с использованием более продвинутых инструментов наташи сделать на самом деле красивую разметку:

Посмотрим, что возвращает разметка Doc:

In [16]:
for text in sents[:2]:
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    display(doc.spans)

[DocSpan(start=11, stop=24, type='ORG', text='Парк культуры', tokens=[...]),
 DocSpan(start=27, stop=32, type='ORG', text='Гараж', tokens=[...]),
 DocSpan(start=96, stop=107, type='PER', text='Сигеру Бана', tokens=[...]),
 DocSpan(start=170, stop=184, type='PER', text='Рэмом Колхасом', tokens=[...])]

[DocSpan(start=19, stop=33, type='ORG', text='Шестигранником', tokens=[...])]

Составим функцию:

In [17]:
def doc_ner(sent):
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    # doc.spans
    prepare_ents = []
    for i in doc.spans:
        prepare_ents.append((
              i.start,
              i.stop,
              i.type
            ))
    ents_dt_mn = list(dates_extractor(sent)) + list(money_extractor(sent))
    for ent in ents_dt_mn: 
        prepare_ents.append((
          ent.start,
          ent.stop,
          ent.fact.__class__.__name__
        ))
    return show_span_line_markup(
        sent,
        prepare_ents
        )

Результат:

In [18]:
for sent in sents:
    doc_ner(sent)